# Import necessary packages if not already installed

In [ ]:
!pip install mlx-lm-lora mlx-lm datasets

# Import your needed modules

In [30]:
from mlx_lm_lora.trainer.sft_trainer import SFTTrainingArgs, train_sft
from mlx_lm_lora.trainer.datasets import CacheDataset, TextDataset

from datasets import load_dataset, concatenate_datasets

from mlx_lm.tuner.callbacks import TrainingCallback
from mlx_lm.utils import load
from mlx_lm.tuner.utils import linear_to_lora_layers, print_trainable_parameters

import mlx.optimizers as optim

# Define the Args

In [31]:
model_name = "mlx-community/Josiefied-Qwen3-0.6B-abliterated-v1-4bit"
new_model_name = "Josiefied-Qwen3-14B-abliterated-v3"
adapter_path = "path/to/adapters"
max_seq_length = 512
num_layers = 12
lora_parameters = {"rank": 8, "dropout": 0.0, "scale": 10.0}

dataset_names = [
    "Goekdeniz-Guelmez/Wizzard-smol"
]
dataset_samples = None

# Load the model

In [ ]:
model, tokenizer = load(model_name)

# Convert to LoRA

In [ ]:
model.freeze()

linear_to_lora_layers(
    model=model,
    num_layers=num_layers,
    config=lora_parameters,
    use_dora=False,
)

print_trainable_parameters(model)

# Define the Optimizer

In [4]:
opt = optim.AdamW(learning_rate=1e-5)

# Load and Preprocess your Dataset

In [26]:
system_prompt = """You are **J.O.S.I.E.**, an advanced super-intelligent AI Assistant created by a 25 year old man named **Gökdeniz Gülmez**."""

def format_prompts_func(sample):
    this_conversation = sample["conversations"]

    if isinstance(this_conversation, list):
        conversation = []
        conversation.append({"role": "system", "content": system_prompt})
        for turn in this_conversation:
            if turn["from"] == "human":
                conversation.append({"role": "user", "content": turn['value']})
            elif turn["from"] == "gpt":
                conversation.append({"role": "assistant", "content": turn['value']})
        
    sample["text"] = tokenizer.apply_chat_template(
        conversation=conversation,
        add_generation_prompt=False,
        enable_thinking=False, # <- Only for Qwen models
        tokenize=False
    )
    return sample

datasets = [load_dataset(name)["train"] for name in dataset_names]
combined_dataset = concatenate_datasets(datasets)

if dataset_samples is not None:
    combined_dataset = combined_dataset.select(range(dataset_samples))

full_dataset = combined_dataset.map(format_prompts_func,)

train_dataset, valid_dataset = full_dataset.train_test_split(test_size=0.01, seed=42).values() # Split the full dataset into 99# Train and 1% Validation

In [ ]:
print(train_dataset[0]["text"])

# 📦 Make the Dataset for the trainer

You have multiple dataset wrappers available depending on your training format:

- **`TextDataset`**: Use this when your data is a plain string under a specific key like `"text"`.
- **`CompletionsDataset`**: Use when you have separate fields like `prompt` and `completion`.
- **`ChatDataset`**: Use this for structured chat data like Alpaca-style, ShareGPT, etc.

Be sure to import the wanted Dataset classes above.

In [ ]:
train_set = TextDataset(train_dataset, tokenizer, text_key='text')
valid_set = TextDataset(train_dataset, tokenizer, text_key='text')

# Start training

In [ ]:
train_sft(
    model=model,
    args=SFTTrainingArgs(
        batch_size=1,
        iters=100,
        val_batches=1,
        steps_per_report=20,
        steps_per_eval=50,
        steps_per_save=50,
        adapter_file="path/to/adapter/file",
        max_seq_length=512,
        grad_checkpoint=True,
    ),
    optimizer=opt,
    train_dataset=CacheDataset(train_set),
    val_dataset=CacheDataset(valid_set),
    training_callback=TrainingCallback()
)

# Fuse the model with the trained adapters and save the new model

In [ ]:
!mlx_lm.fuse --model mlx-community/Josiefied-Qwen3-0.6B-abliterated-v1-4bit --adapter-path path/to/save/adaper --save-path path/to/new/model --upload-repo mlx-comunity/my-new-model